In [13]:
using Statistics
using TextAnalysis

# ファイルを読み込む
file_path = "./data/20231102_alice.txt"
text = read(file_path, String)

# テキストを小文字に変換
text = lowercase(text)
doc = StringDocument(text)
# prepare!(doc, strip_punctuation)
# テキストを文に分割する

A StringDocument{String}
 * Language: Languages.English()
 * Title: Untitled Document
 * Author: Unknown Author
 * Timestamp: Unknown Time
 * Snippet: alice was beginning to get very tired of sitting b

In [14]:
include("./lib/expand_contractions.jl")
text = expand_contractions(text)

"alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “wi" ⋯ 148718 bytes ⋯ "yes bright and eager with many a strange tale, perhaps even with the dream of wonderland of long ago: and how she would feel with all their simple sorrows, and find a pleasure in all their simple joys, remembering her own child-life, and the happy summer days.\n\nthe end"

In [40]:
sentences = split(text, r"[.?!“”(),\n]+")

4861-element Vector{SubString{String}}:
 "alice was beginning to get very tired of sitting by her sister on the bank"
 " and of having nothing to do: o" ⋯ 33 bytes ⋯ "the book her sister was reading"
 " but it had no pictures or conversations in it"
 " "
 "and what is the use of a book"
 " thought alice "
 "without pictures or conversations"
 "so she was considering in her own mind "
 "as well as she could"
 " for the hot day made her feel very sleepy and stupid"
 " whether the pleasure of making" ⋯ 48 bytes ⋯ "ting up and picking the daisies"
 " when suddenly a white rabbit with pink eyes ran close by her"
 "there was nothing so very remar" ⋯ 67 bytes ⋯ "o hear the rabbit say to itself"
 ⋮
 "lastly"
 " she pictured to herself how this same little sister of hers would"
 " in the after-time"
 " be herself a grown woman; and how she would keep"
 " through all her riper years"
 " the simple and loving heart of" ⋯ 41 bytes ⋯ "about her other little children"
 " and make their eyes bright a

In [41]:

# 各文を単語に分割し、STARTとENDを追加する
processed_sentences = []
for sentence in sentences
    words = split(sentence)
    if isempty(words) || length(words) < 3
        continue
    end
    pushfirst!(words, "START")
    push!(words, "END")
    push!(processed_sentences, words)
end

In [46]:
# バイグラムを生成する (文境界を含むものを除去)
bigrams = []
for sentence in processed_sentences
    for i in 1:length(sentence)-1
        bigram = (sentence[i], sentence[i+1])
        if sentence[i] != "END" && sentence[i+1] != "START"
            push!(bigrams, bigram)
        end
    end
end

# バイグラムの出現回数を集計する
bigram_counts = Dict()
for bigram in bigrams
    bigram_counts[bigram] = get(bigram_counts, bigram, 0) + 1
end

# 出力して確認
bigram_counts

Dict{Any, Any} with 12999 entries:
  ("doubtfully:", "END")     => 1
  ("uncomfortably", "sharp") => 1
  ("salt", "water")          => 2
  ("is", "to")               => 6
  ("at", "them")             => 3
  ("why", "END")             => 3
  ("simple", "and")          => 1
  ("crazy", "END")           => 1
  ("the", "bones")           => 1
  ("‘coming", "in")          => 1
  ("a", "lobster")           => 2
  ("footman", "in")          => 2
  ("shall", "only")          => 1
  ("corner—no", "END")       => 1
  ("one", "only")            => 1
  ("a", "delightful")        => 1
  ("in", "one")              => 3
  ("that", "curious")        => 1
  ("a", "present")           => 1
  ("solemnly", "END")        => 2
  ("they", "hit")            => 1
  ("START", "till")          => 4
  ("they", "sat")            => 2
  ("going", "through")       => 1
  ("get", "out")             => 5
  ⋮                          => ⋮

In [43]:
p_sentences = []

for sentence in processed_sentences
    push!(p_sentences, join(sentence, " "))
end

# join sentences


processed_text = join(p_sentences, "\n")

output = "./data/alice_processed.txt"
open(output, "w") do f
    write(f, processed_text)
end

163047

In [86]:
include("./lib/calc_entropy.jl")

# 指定された文
case1 = ["START is the rabbit on the table white END", "START does the rabbit on the table red END"]
case2 = ["START is the girl reading END", "START does the rabbit reading END"]
case3 = ["START is the caterpillar tired END", "START does the caterpillar tired END"]
case4 = ["START is the house old END", "does the house old END"]

function compare_2sentences(sentence1, sentence2)
    words1 = split(sentence1)
    words2 = split(sentence2)
    bigrams1 = [(words1[i], words1[i+1]) for i in 1:length(words1)-1]
    cross_entropy1 = calc_entropy(bigrams1, bigram_counts)
    bigrams2 = [(words2[i], words2[i+1]) for i in 1:length(words2)-1]
    cross_entropy2 = calc_entropy(bigrams2, bigram_counts)
    println("Cross Entropy: ", cross_entropy1, " ", cross_entropy2)
end
compare_2sentences(case1[1], case1[2])
compare_2sentences(case2[1], case2[2])
compare_2sentences(case3[1], case3[2])
compare_2sentences(case4[1], case4[2])

Cross Entropy: (5.999290282296896, Any[("START", "is", 5), ("is", "the", 30), ("the", "rabbit", 20), ("rabbit", "on", 0), ("on", "the", 34), ("the", "table", 8), ("table", "white", 0), ("white", "END", 1)]) (6.452332898918942, Any[("START", "does", 2), ("does", "the", 2), ("the", "rabbit", 20), ("rabbit", "on", 0), ("on", "the", 34), ("the", "table", 8), ("table", "red", 0), ("red", "END", 1)])
Cross Entropy: (5.15852213041289, Any[("START", "is", 5), ("is", "the", 30), ("the", "girl", 0), ("girl", "reading", 0), ("reading", "END", 1)]) (7.337793592041413, Any[("START", "does", 2), ("does", "the", 2), ("the", "rabbit", 20), ("rabbit", "reading", 0), ("reading", "END", 1)])
Cross Entropy: (4.522911364343301, Any[("START", "is", 5), ("is", "the", 30), ("the", "caterpillar", 24), ("caterpillar", "tired", 0), ("tired", "END", 0)]) (5.247779550938574, Any[("START", "does", 2), ("does", "the", 2), ("the", "caterpillar", 24), ("caterpillar", "tired", 0), ("tired", "END", 0)])
Cross Entropy: (

In [114]:
include("./lib/biagram_export.jl")
# バイグラムのデータセット
bigram_data = [
    ("START", "Alice", 5),
    ("Alice", "was", 15),
    ("was", "beginning", 10),
    ("beginning", "to", 8),
    ("to", "get", 12),
    ("get", "very", 7),
    ("very", "tired", 3),
    ("tired", "of", 4),
    ("of", "sitting", 2),
    ("sitting", "by", 6),
    ("by", "her", 9),
    ("her", "sister", 11),
    ("sister", "on", 13),
    ("on", "the", 14),
    ("the", "bank", 16),
    ("bank", "END", 1)
]

# 関数を呼び出してグラフを生成し、ファイルに保存
generate_bigram_graph(bigram_data, "sample_graph")

["START Alice", "Alice was", "was beginning", "beginning to", "to get", "get very", "very tired", "tired of", "of sitting", "sitting by", "by her", "her sister", "sister on", "on the", "the bank", "bank END"]

"C:\\Users\\shion\\gitClones\\JuliaAliceBigram\\sample_graph.png"